In [ ]:
import os
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy
# plt.style.use("ggplot")
%matplotlib inline

# from tqdm import tqdm_notebook, tnrange
from itertools import chain
from skimage.io import imread, imshow, concatenate_images
from skimage.transform import resize
from skimage.morphology import label
from sklearn.model_selection import train_test_split

import tensorflow as tf

from keras.models import Model, load_model
from keras.layers import Input, BatchNormalization, Activation, Dense, Dropout
from keras.layers.core import Lambda, RepeatVector, Reshape
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D, GlobalMaxPool2D
from keras.layers.merge import concatenate, add
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.optimizers import Adam
from tensorflow.keras import backend as K
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [ ]:
im_width = 128
im_height = 128
border = 5
path_data = '../../../Desktop/training9/img/'
path_label = '../../../Desktop/training9/label/'
train_num = 590

In [ ]:
# Get and resize train images and masks

x = np.zeros((train_num, im_height, im_width, 1), dtype=np.float32)
y = np.zeros((train_num, im_height, im_width, 1), dtype=np.float32)

index = 0
for fname in os.listdir(path_data):
    img = load_img(path_data + fname, grayscale=True)
    x_img = img_to_array(img)
    x_img = resize(x_img, (128, 128, 1), mode='constant', preserve_range=True)

    img = load_img(path_label + fname, grayscale=True)
    y_img = img_to_array(img)
    y_img = resize(y_img, (128, 128, 1), mode='constant', preserve_range=True)
    
    
    # Save images
    x[index, ..., 0] = x_img.squeeze() / 255
    y[index] = y_img / 255
    
    index += 1

In [ ]:
im_width = 128
im_height = 128
border = 5
path_data = '../../../Desktop/new5/img/'
path_label = '../../../Desktop/new5/label/'
predict_len = 131

x_new = np.zeros((predict_len, im_height, im_width, 1), dtype=np.float32)
y_new = np.zeros((predict_len, im_height, im_width, 1), dtype=np.float32)

index = 0
for fname in os.listdir(path_data):
    img = load_img(path_data + fname, grayscale=True)
    x_img = img_to_array(img)
    x_img = resize(x_img, (128, 128, 1), mode='constant', preserve_range=True)

    img = load_img(path_label + fname, grayscale=True)
    y_img = img_to_array(img)
    y_img = resize(y_img, (128, 128, 1), mode='constant', preserve_range=True)
    
    
    # Save images
    x_new[index, ..., 0] = x_img.squeeze() / 255
    y_new[index] = y_img / 255
    
    index += 1

In [ ]:
# Split train and valid
x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size=0.15, random_state=2018)

In [ ]:
def conv2d_block(input_tensor, n_filters, kernel_size=3, batchnorm=True):
    # first layer
    x = Conv2D(filters=n_filters, kernel_size=(kernel_size, kernel_size), kernel_initializer="he_normal",
               padding="same")(input_tensor)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation("relu")(x)
    # second layer
    x = Conv2D(filters=n_filters, kernel_size=(kernel_size, kernel_size), kernel_initializer="he_normal",
               padding="same")(x)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation("relu")(x)
    return x

In [ ]:
def get_unet(input_img, n_filters=16, dropout=0.5, batchnorm=True):
    # contracting path
    c1 = conv2d_block(input_img, n_filters=n_filters*1, kernel_size=3, batchnorm=batchnorm)
    p1 = MaxPooling2D((2, 2)) (c1)
    p1 = Dropout(dropout*0.5)(p1)

    c2 = conv2d_block(p1, n_filters=n_filters*2, kernel_size=3, batchnorm=batchnorm)
    p2 = MaxPooling2D((2, 2)) (c2)
    p2 = Dropout(dropout)(p2)

    c3 = conv2d_block(p2, n_filters=n_filters*4, kernel_size=3, batchnorm=batchnorm)
    p3 = MaxPooling2D((2, 2)) (c3)
    p3 = Dropout(dropout)(p3)

    c4 = conv2d_block(p3, n_filters=n_filters*8, kernel_size=3, batchnorm=batchnorm)
    p4 = MaxPooling2D(pool_size=(2, 2)) (c4)
    p4 = Dropout(dropout)(p4)
    
    c5 = conv2d_block(p4, n_filters=n_filters*16, kernel_size=3, batchnorm=batchnorm)
    
    # expansive path
    u6 = Conv2DTranspose(n_filters*8, (3, 3), strides=(2, 2), padding='same') (c5)
    u6 = concatenate([u6, c4])
    u6 = Dropout(dropout)(u6)
    c6 = conv2d_block(u6, n_filters=n_filters*8, kernel_size=3, batchnorm=batchnorm)

    u7 = Conv2DTranspose(n_filters*4, (3, 3), strides=(2, 2), padding='same') (c6)
    u7 = concatenate([u7, c3])
    u7 = Dropout(dropout)(u7)
    c7 = conv2d_block(u7, n_filters=n_filters*4, kernel_size=3, batchnorm=batchnorm)

    u8 = Conv2DTranspose(n_filters*2, (3, 3), strides=(2, 2), padding='same') (c7)
    u8 = concatenate([u8, c2])
    u8 = Dropout(dropout)(u8)
    c8 = conv2d_block(u8, n_filters=n_filters*2, kernel_size=3, batchnorm=batchnorm)

    u9 = Conv2DTranspose(n_filters*1, (3, 3), strides=(2, 2), padding='same') (c8)
    u9 = concatenate([u9, c1], axis=3)
    u9 = Dropout(dropout)(u9)
    c9 = conv2d_block(u9, n_filters=n_filters*1, kernel_size=3, batchnorm=batchnorm)
    
    outputs = Conv2D(1, (1, 1), activation='sigmoid') (c9)
    model = Model(inputs=[input_img], outputs=[outputs])
    return model

In [ ]:
def dice_loss(y_true, y_pred, smooth=1e-12):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)

    return 1-K.mean( (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth) )
# def dice_loss(y_true,y_pred):
#     numerator = 2 * tf.reduce_sum(y_true * y_pred, axis=-1)
#     denominator = tf.reduce_sum(y_true + y_pred, axis=-1)

#     return 1 - (numerator + 1) / (denominator + 1)

In [ ]:
input_img = Input((im_height, im_width, 1), name='img')
model = get_unet(input_img, n_filters=16, dropout=0.05, batchnorm=True)

model.compile(optimizer=Adam(), loss=dice_loss, metrics=["accuracy"])

In [ ]:
callbacks = [
    EarlyStopping(patience=10, verbose=1),
    ReduceLROnPlateau(factor=0.1, patience=3, min_lr=0.00001, verbose=1),
    ModelCheckpoint('model-tgs-salt.h5', verbose=1, save_best_only=True, save_weights_only=True)
]

In [ ]:
# results = model.fit(x_train, y_train, batch_size=32, epochs=100, callbacks=callbacks, validation_split=0.3)

results = model.fit(x_train, y_train, batch_size=32, epochs=100, callbacks=callbacks, 
                    validation_data=(x_valid,y_valid))

In [ ]:
plt.figure(figsize=(8, 8))
plt.title("Learning curve")
plt.plot(results.history["loss"], label="loss")
plt.plot(results.history["val_loss"], label="val_loss")
plt.plot( np.argmin(results.history["val_loss"]), np.min(results.history["val_loss"]), marker="x", color="r", label="best model")
plt.xlabel("Epochs")
plt.ylabel("log_loss")
plt.legend();

In [ ]:
# Load best model
model.load_weights('model-tgs-salt.h5')

In [ ]:
# Evaluate on validation set (this must be equals to the best log_loss)
# model.evaluate(X_valid, y_valid, verbose=1)
model.evaluate(x_new, y_new, verbose=1)

In [ ]:
preds_val   = model.predict(x_new, verbose=1)

In [ ]:
def plot_sample(X, y, preds, ix=None):
    #squeeze() loss one dimension (128,128,1) (128,128)      #softmax use exp() to become[0,1]
    output = preds[ix].squeeze()  #     output = softmax(preds[ix].squeeze())*255
    new = []
    for i in range(128):
        new.append([])    
        for j in range(128):
            new[i].append(output[i][j])
            
            
    fig, ax = plt.subplots(1, 4, figsize=(20, 10))
    ax[0].imshow(X[ix, ..., 0],cmap='gray')
    ax[0].set_title('ILD')
    
    out_y = y[ix].squeeze()
    h = np.dstack([X[ix, ..., 0], X[ix, ..., 0], X[ix, ..., 0]])
    for i in range(128):
        for j in range(128):
            if out_y[i][j] != 0:
                h[i, j, :] = [1, 0, 0]
    

    ax[1].set_title('LABEL')
    ax[1].imshow(X[ix, ..., 0],cmap='gray')
    ax[1].imshow(h,alpha=0.5)

    

    ax[2].imshow(new)
    ax[2].set_title("direct predict")
    
    I = np.dstack([X[ix, ..., 0], X[ix, ..., 0], X[ix, ..., 0]])
    for i in range(128):
        for j in range(128):
            new[i][j] = sigmoid(new[i][j])
            if new[i][j] > 0.53:
                new[i][j] = 255
                I[i, j, :] = [1, 0, 0]
            else:
                new[i][j] = 0
                
    ax[3].set_title('overlap')    
    ax[3].imshow(X[ix, ..., 0],cmap='gray')
    ax[3].imshow(I,alpha=0.5)
    

#     fig.savefig('../../output_pic_unet2/'+str(ix)+'.png')
    

In [ ]:
import math
def sigmoid(x):
    return 1 / (1 + math.exp(-x))
for i in range(predict_len):
    plot_sample(x_new, y_new, preds_val,ix=i)


In [ ]:
# calculate the dice 
for k in range(500,640,10):
    dice_thre = k/1000
    total = 0
    count = 0
    for i in range(predict_len):
        white_sum = 0
        white_overlap = 0
        Dice = 0

        oriy  = y_new[i].squeeze()
        predy = preds_val[i].squeeze()

        new_oriy = []
        for a in range(128):
            new_oriy.append([])    
            for b in range(128):
                if(  oriy[a][b]  > 0.03 ):
                    new_oriy[a].append(1)
                else:
                    new_oriy[a].append(0)

        new_predy = []
        for a in range(128):
            new_predy.append([])
            for b in range(128):
                if(sigmoid(predy[a][b])>dice_thre):
                    new_predy[a].append(1)
                else:
                    new_predy[a].append(0)
        

        for j in range(128):
            for k in range(128):
                if new_oriy[j][k]:
                    white_sum+=1
                if new_predy[j][k]:
                    white_sum+=1
                if new_oriy[j][k] and new_predy[j][k]:
                    white_overlap+=1

        if white_sum != 0:
            Dice = (2*white_overlap) / white_sum
        else:
            Dice=0

        if(Dice!=0):
            total+=Dice
            count+=1

    print("dice_threshold: ",end="")
    print(dice_thre,end="  ")
    print("average Dice: ",end=" ")
    if count == 0:
        print("{:.2f}".format(0))
    else:
        print("{:.2f}".format(total/count))
    print()


In [ ]:
# # calculate the dice 
# for k in range(530,570,1):
#     dice_thre = k/1000
#     total = 0
#     count = 0
#     for i in range(predict_len):
#         white_sum = 0
#         white_overlap = 0
#         Dice = 0

#         oriy  = y_new[i].squeeze()
#         predy = preds_val[i].squeeze()

#         new_oriy = []
#         for a in range(128):
#             new_oriy.append([])    
#             for b in range(128):
#                 if(  oriy[a][b]  > 0.03 ):
#                     new_oriy[a].append(1)
#                 else:
#                     new_oriy[a].append(0)

#         new_predy = []
#         for a in range(128):
#             new_predy.append([])
#             for b in range(128):
#                 if(sigmoid(predy[a][b])>dice_thre):
#                     new_predy[a].append(1)
#                 else:
#                     new_predy[a].append(0)


#         for j in range(128):
#             for k in range(128):
#                 if new_oriy[j][k]:
#                     white_sum+=1
#                 if new_predy[j][k]:
#                     white_sum+=1
#                 if new_oriy[j][k] and new_predy[j][k]:
#                     white_overlap+=1

#         if white_sum != 0:
#             Dice = (2*white_overlap) / white_sum
#         else:
#             Dice=0

#         if(Dice!=0):
#             total+=Dice
#             count+=1

#     print("dice_threshold: ",end="")
#     print(dice_thre,end="  ")
#     print("average Dice: ",end=" ")
#     if count == 0:
#         print("{:.2f}".format(0))
#     else:
#         print("{:.2f}".format(total/count))
#     print()